# Wildberries Scraper

Попытка автоматизировать получение информации о товарах на WB. 
<br>Изображения хранятся в /res/img
<br>data.csv - список ссылок, полученный из выдачи WB
<br>metadata.csv - Информация о конкретном товаре, его описание, метаданные обложки карточки товара (размер, путь, расширение и т.п.)*

<br><i>* Поскольку WB хранит изображения на отдельных серверах (другой домен ~wbbasket.ru), из-за правила 'same-origin policy' изображения нельзя побитово сохранить с помощью JavaScript, также при использовании request с разными хедерами запрос блокируется спустя несколько итераций => для данного способа нужны прокси, а wb определяет использование прокси, соответственно бесплатно эту проблему не решить. Пришлось, как и в случае с LinkedIn, обходить это ограничение с помощью скриншотов</i>

<br><i>В файле metadata.csv есть несколько лишних строк (в начале), которые относятся к тестовым запускам скрапера. Не стал их удалять из самого файла. В конце ноутбука есть удаление дубликатов из датафрейма</i>

### Функции перехода по страницам и получения ссылок карточек выдачи WB

In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm

import os
import csv
from bs4 import BeautifulSoup

from PIL import Image

In [40]:
def write_metadata(metadata: dict, filename: str):
    csv_file_path = filename

    if not os.path.exists(csv_file_path):
        with open(csv_file_path, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=metadata.keys())
            writer.writeheader()

    with open(csv_file_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=metadata.keys())
        writer.writerow(metadata)

In [15]:
def go_next_page(driver, page_count: int):
    next_page = driver.find_elements(By.CLASS_NAME, ('j-next-page'))
    if next_page:
        page_count += 1
        driver.get(f'https://www.wildberries.ru/catalog/igrushki/igrushechniy-transport?sort=popular&page={page_count}')
    return page_count

def get_wb_product_links(driver):
    items_count = 0
    cards = driver.find_elements(By.CLASS_NAME, "product-card__link")
    for card in cards:
        title = card.get_attribute('aria-label')
        url = card.get_attribute('href')
        temp_data = {
            'title' : title,
            'url' : url
        }
        items_count += 1 
        write_metadata(temp_data, 'data.csv')
    return items_count

### Функция, имитирующая естественную прокрутку в конец страницы 
(для того, чтобы загрузился динамический контент, затем скролл вверх, чтобы стала видна кнопка следующей страницы)

In [16]:
def scroll_to_bottom(driver):
    page_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_page_height = driver.execute_script("return document.body.scrollHeight")
    if new_page_height != page_height:
        scroll_to_bottom(driver)
    time.sleep(1)
    # Scrolling up a little bit
    driver.execute_script("window.scrollBy(0, -250);")

### Функции для скрапинга страницы товара и записи в файл

In [17]:
import csv

def scrape_wb_page(driver, url):
    driver.get(url)
    time.sleep(5)
    
    # Text data
    product_art = driver.find_element(By.ID, 'productNmId').text
    product_title = driver.find_element(By.CSS_SELECTOR, 'h1.product-page__title').text
    product_description = driver.find_element(By.CSS_SELECTOR, 'table.product-params__table').get_attribute("outerHTML")
    
    # Cover image retrieval    
    image_elements = driver.find_elements(By.XPATH, '//img[contains(@alt, "Вид 1")]')
    image_url = image_elements[0].get_attribute("src")
    
    driver.get(image_url)
    time.sleep(2)
    img = driver.find_element(By.TAG_NAME, 'img')

    image_path = f'res/img/{product_art}.png'
    with open(image_path, 'wb') as file:
        file.write(img.screenshot_as_png)
        file.close()

    # Getting image metadata
    
    image = Image.open(image_path)
    filesize = os.path.getsize(image_path) / 1024 # Converting to KB
    
    metadata = {
    "id": int(product_art),
    "product_title": product_title,
    "product_description": BeautifulSoup(product_description, 'html.parser').get_text(),
    "filename": image_path,
    "format": image.format,
    "mode": image.mode,
    "resolution": image.size,
    "filesize": round(filesize, 2),
    "retrieved": time.time()
    }
    
    image.close()

    
    return metadata

def write_metadata(metadata: dict, filename: str):
    csv_file_path = filename

    if not os.path.exists(csv_file_path):
        with open(csv_file_path, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=metadata.keys())
            writer.writeheader()

    with open(csv_file_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=metadata.keys())
        writer.writerow(metadata)
    

### Получение ссылок из выдачи WB

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome()


#a.pagination-next.pagination__next.j-next-page


page_url = 'https://www.wildberries.ru/catalog/igrushki/igrushechniy-transport?sort=popular&page=1'
page_count = 1
items_count = 0
target = 350

driver.get(page_url)

with tqdm(total = target) as pbar:
    while (items_count < target):
        time.sleep(5)
        scroll_to_bottom(driver)
        items_temp = get_wb_product_links(driver)
        print(f'Retrieved {items_temp} items')
        pbar.update(items_temp)
        items_count += items_temp
        try:
            page_count = go_next_page(driver, page_count)
        except: 
            break

driver.quit()

 13%|█▎        | 45/350 [00:34<03:53,  1.31it/s]

Retrieved 45 items


 13%|█▎        | 45/350 [00:43<04:57,  1.03it/s]


### Загрузка ссылок в датафрейм
Взял с запасом, на случай возникновения ошибок во время скрапинга

In [38]:
import pandas as pd

data = pd.read_csv('data.csv')
data = data[450:700]

In [39]:
data

,title,url
450,Машинка прозрачная для детей Машинка Детская,https://www.wildberries.ru/catalog/21277143/de...
451,Игрушка коллекционная машинка металлическая мо...,https://www.wildberries.ru/catalog/151278736/d...
452,Машинка игрушка для мальчиков Коллекционная ме...,https://www.wildberries.ru/catalog/20849619/de...
453,Игрушечный транспорт Пожарная машина Autoprofi,https://www.wildberries.ru/catalog/176134492/d...
454,Металлическая машинка Тесла с трейлером и квад...,https://www.wildberries.ru/catalog/178158637/d...
...,...,...
695,Игрушка детская машинка металлическая Chevy 51...,https://www.wildberries.ru/catalog/137626586/d...
696,Подводная лодка с торпедами YourLOVElyHome,https://www.wildberries.ru/catalog/140413919/d...
697,Игрушка детская машинка металлическая модель B...,https://www.wildberries.ru/catalog/62000996/de...
698,Игрушка детская машинка железная Lamborghini H...,https://www.wildberries.ru/catalog/150656570/d...


### Скрапинг страниц товаров

Одна итерация с ошибкой - не успели загрузиться некоторые html элементы
<br>227 из 228 успешно

In [50]:
from tqdm import tqdm
from IPython.display import clear_output
import warnings

try:
    data = pd.read_csv('data.csv')
    data = data[450:700]
except: 
    raise FileNotFoundError(f"data.csv not found")

driver.quit()
driver = webdriver.Chrome()
driver.get('https://wildberries.ru/')


for _, row in tqdm(data.iterrows(), desc='Processing'):
    #driver = webdriver.Chrome()
    print(f"Working on: {row['title']}")
    print(row['url'])
    #try:
    metadata = scrape_wb_page(driver, row['url'])
    write_metadata(metadata, 'metadata.csv')
    #except:
        #warnings.warn(f"Failed to save {row['title']}")
        #continue
    clear_output(wait=True)
    


Processing: 204it [29:01,  8.22s/it]

Working on: Автозаправка с автомойкой. 17*27*13 см КубиГрад
https://www.wildberries.ru/catalog/188749568/detail.aspx


Processing: 204it [29:07,  8.57s/it]


KeyboardInterrupt: 

In [51]:
metadata_csv = pd.read_csv('metadata.csv')
metadata_csv['retrieved'] = pd.to_datetime(metadata_csv['retrieved'], unit='s')
metadata_csv['file_exists'] = metadata_csv['filename'].apply(os.path.exists)
print(f'Rows count: {len(metadata_csv)}\nImages exist: {metadata_csv["file_exists"].sum()}')
metadata_csv

Rows count: 440
Images exist: 440


,id,product_title,product_description,filename,format,mode,resolution,filesize,retrieved,file_exists
0,209650955,Триммер садовый электрический для травы,Артикул 209650955 Модель беспроводной; аккуму...,res/img/209650955.png,PNG,RGBA,"(246, 328)",171.35,2024-03-21 11:10:04.299254016,True
1,209650955,Триммер садовый электрический для травы,Артикул 209650955 Модель беспроводной; аккуму...,res/img/209650955.png,PNG,RGBA,"(246, 328)",171.35,2024-03-21 11:13:28.587107072,True
2,77278318,Триммер аккумуляторный садовый дачный кусторез,Артикул 77278318 Модель разборная модель Пита...,res/img/77278318.png,PNG,RGBA,"(246, 328)",160.00,2024-03-21 11:13:44.241293056,True
3,18565737,Триммер аккумуляторный ZITREK GreenCut 20 (20В...,Артикул 18565737 Модель GreenCut 20 Гарантийн...,res/img/18565737.png,PNG,RGBA,"(246, 328)",17.67,2024-03-21 11:13:58.767094016,True
4,113066515,Триммер аккумуляторный садовый дачный кусторез,Артикул 113066515 Модель разборная модель Пит...,res/img/113066515.png,PNG,RGBA,"(246, 328)",128.90,2024-03-21 11:14:13.620366080,True
...,...,...,...,...,...,...,...,...,...,...
435,25588413,"Транспорт медицинской службы KiddieDrive, 8см",Артикул 25588413 Материал игрушки пластик; ме...,res/img/25588413.png,PNG,RGBA,"(246, 328)",46.80,2024-05-08 15:23:05.902631936,True
436,140075474,Детская игрушка машинка эвакуатор,Артикул 140075474 Количество предметов в упак...,res/img/140075474.png,PNG,RGBA,"(246, 328)",164.06,2024-05-08 15:23:14.496779008,True
437,117591684,Игрушка детская машинка металлическая Lada 210...,Артикул 117591684 Количество предметов в упак...,res/img/117591684.png,PNG,RGBA,"(246, 328)",34.21,2024-05-08 15:23:22.821166080,True
438,149041429,Бэтмобиль,Артикул 149041429 Количество предметов в упак...,res/img/149041429.png,PNG,RGBA,"(246, 328)",117.40,2024-05-08 15:23:30.875238912,True


### Избавляемся от следов предыдущих запусков

In [57]:
metadata_csv = metadata_csv.drop_duplicates(subset='id')

In [58]:
metadata_csv.to_csv('metadata.csv')